In [ ]:
import pandas as pd
import numpy as np
import json
import nltk

In [ ]:
df = pd.read_json("../data/marmiton.json")

In [ ]:
df.head()

In [ ]:
recipies = df[["title", "ingredients"]]

In [ ]:
test = recipies.ingredients

In [ ]:
mesures = np.array(['dl', 'l', 'ml', 'cl', 'g', 'kg', 'cuillère', 'cuillères', 'cuillerées', 'pincée', 'poignée', 'pincées', 'verre'])
litre_ml = ["ml","cl","dl","l"]
poids_g = ["g", "kg"]
long = np.array(['cuillère', 'cuillères', 'cuillerées'])
long_num = np.array([3, 3, 3])
stop_word = np.array(['une', 'de', 'petits', 'petites', 'petite', 'petit', 'grand', 'gros', 'grosse', 'beaux', 'beau'])
mult = np.array(['ou', 'à'])
quant = np.array(['paquet', 'sachet'])

In [ ]:
def number(s):
    s = s.replace(',', '.')
    try:
        return float(s)
    except ValueError:
        if '/' in s and s[0].isdigit():
            a, b = s.split('/')
            
            if not number(a) or not number(b):
                return False
            
            return float(a) / float(b)
        else:
            return False

In [ ]:
np.where([False, False, True, False, False])

In [ ]:
def parse(ingredients):
    res = []
    for ingredient in ingredients:
        arr = np.array(nltk.word_tokenize(ingredient))
        arr = arr[np.in1d(arr, stop_word, invert=True)]
        
        if len(arr) > 0:

            quant = '/'
            mesure = "/"

            num = number(arr[0])

            if len(arr) > 1 and num:
                quant = num

                if arr[1] in mult:
                    arr = arr[2:]

                ingredient = ""

                num = 0

                if arr[1].lower() in mesures:
                    mesure = arr[1].lower()
                    num = 1

                    if mesure in long:
                        num = long_num[long == mesure]
                        mesure = " ".join(arr[1:1+num])
                
                if mesure in litre_ml:
                    quant *= 10 ** np.where(litre_ml == mesure)[0]
                    
                if mesure in poids_g:
                    quant *= 1000 ** np.where(poids_g == mesure)[0]
                    
                arr = arr[1+num:]
            elif arr[0][0].isdigit():
                _1 = arr[0]
                for mes in mesures:
                    if mes in _1:
                        quant = _1.replace(mes, '')
                        mesure = mes
                        arr[1:]

            if '(' in arr:
                idx = np.where(arr == '(')[0]
                if len(idx) > 1:
                    idx = idx[0]
                arr = arr[:idx]

            res.append((quant, mesure, " ".join(arr)))
        
    return res

In [ ]:
res = [parse(ingds) for ingds in test]